# -------------------------------BESIC SETUP AND TESTING-----------------------------------

In [1]:
import os
os.chdir("/Users/naveenkumar/Desktop/formula-1-bot")
%pwd

'/Users/naveenkumar/Desktop/formula-1-bot'

In [2]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("Dependencies imported successfully!")

Dependencies imported successfully!


In [3]:
# Database configuration from environment variables
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'f1_bot')
DB_USER = os.getenv('DB_USER', 'f1_user')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'f1_password')

# Create SQLAlchemy engine
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(DATABASE_URL)
    # Test connection
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        print("✅ Database connection successful!")
        print(f"PostgreSQL version: {result.fetchone()[0]}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("Please check your environment variables (DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)")

✅ Database connection successful!
PostgreSQL version: PostgreSQL 14.18 (Homebrew) on aarch64-apple-darwin24.4.0, compiled by Apple clang version 17.0.0 (clang-1700.0.13.3), 64-bit


In [4]:
# Initialize SQLDatabase for LangChain
db = SQLDatabase(engine)

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

# Create SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

print("✅ LangChain components initialized successfully!")

✅ LangChain components initialized successfully!


In [5]:
# Create SQL agent with custom prompt
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

print("✅ SQL Agent created successfully!")

✅ SQL Agent created successfully!


In [6]:
# First, let's check what's actually in the database
import psycopg2
from sqlalchemy import inspect

# Test direct connection
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn.cursor()
    
    # Get all tables using direct SQL
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    
    tables_direct = cursor.fetchall()
    print("�� Tables found using direct SQL:")
    for table in tables_direct:
        print(f"  - {table[0]}")
    
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"❌ Direct connection error: {e}")

# Check SQLAlchemy inspector
try:
    inspector = inspect(engine)
    tables_inspector = inspector.get_table_names()
    print(f"\n�� Tables found using SQLAlchemy inspector: {tables_inspector}")
    
except Exception as e:
    print(f"❌ Inspector error: {e}")

# Check what db.get_table_names() returns
try:
    tables_langchain = db.get_table_names()
    print(f"\n�� Tables found using LangChain: {tables_langchain}")
    print(f"Type: {type(tables_langchain)}")
    print(f"Length: {len(tables_langchain)}")
    
except Exception as e:
    print(f"❌ LangChain get_table_names error: {e}")

�� Tables found using direct SQL:
  - drivers
  - drivers_transformed
  - intervals
  - intervals_transformed
  - laps
  - laps_transformed
  - meetings
  - pit_stops
  - pit_stops_transformed
  - positions
  - positions_transformed
  - race_control
  - sessions
  - sessions_transformed
  - stints
  - stints_transformed
  - weather
  - weather_transformed

�� Tables found using SQLAlchemy inspector: ['laps_transformed', 'meetings', 'sessions', 'drivers', 'laps', 'pit_stops', 'stints', 'positions', 'weather', 'intervals', 'race_control', 'pit_stops_transformed', 'stints_transformed', 'positions_transformed', 'intervals_transformed', 'weather_transformed', 'drivers_transformed', 'sessions_transformed']

�� Tables found using LangChain: ['drivers', 'drivers_transformed', 'intervals', 'intervals_transformed', 'laps', 'laps_transformed', 'meetings', 'pit_stops', 'pit_stops_transformed', 'positions', 'positions_transformed', 'race_control', 'sessions', 'sessions_transformed', 'stints', 'stin

# -------------------------------DEFINE INTENT IDENTIFIER FUNCTIONS-------------------------------

In [7]:
import re
from typing import Dict, Any, Optional, Tuple
from dataclasses import dataclass
from difflib import get_close_matches

@dataclass
class F1QueryIntent:
    """Structured representation of F1 query intent"""
    query_type: str
    team: Optional[str] = None
    driver1: Optional[str] = None
    driver2: Optional[str] = None
    meeting: Optional[str] = None
    lap_number: Optional[int] = None
    session_type: str = "Race"
    year: Optional[int] = None

class F1QueryValidator:
    """Service class for validating F1 queries and race availability"""
    
    def __init__(self):
        # 2025 F1' Season race mappings
        self.race_keywords = {
            'australian': 'Australian Grand Prix',
            'chinese': 'Chinese Grand Prix',
            'china': 'Chinese Grand Prix',
            'japanese': 'Japanese Grand Prix',
            'japan': 'Japanese Grand Prix',
            'bahrain': 'Bahrain Grand Prix',
            'saudi': 'Saudi Arabian Grand Prix',
            'saudi arabian': 'Saudi Arabian Grand Prix',
            'miami': 'Miami Grand Prix',
            'italian emilia-romagna': 'Emilia‑Romagna Grand Prix',
            'emilia romagna': 'Emilia‑Romagna Grand Prix',
            'emilia-romagna': 'Emilia‑Romagna Grand Prix',
            'imola': 'Emilia‑Romagna Grand Prix',
            'monaco': 'Monaco Grand Prix',
            'spanish': 'Spanish Grand Prix',
            'spain': 'Spanish Grand Prix',
            'canadian': 'Canadian Grand Prix',
            'canada': 'Canadian Grand Prix', 
            'austrian': 'Austrian Grand Prix',
            'austria': 'Austrian Grand Prix',
            'british': 'British Grand Prix',
            'silverstone': 'British Grand Prix',
            'great britain': 'British Grand Prix'
        }
        
        # Session type mappings
        self.session_keywords = {
            'practice': 'Practice',
            'p1': 'Practice 1',
            'practice 1': 'Practice 1',
            'fp1': 'Practice 1',
            'free practice 1': 'Practice 1',
            'p2': 'Practice 2',
            'practice 2': 'Practice 2',
            'fp2': 'Practice 2',
            'free practice 2': 'Practice 2',
            'p3': 'Practice 3',
            'practice 3': 'Practice 3',
            'fp3': 'Practice 3',
            'free practice 3': 'Practice 3',
            'qualifying': 'Qualifying',
            'qualify': 'Qualifying',
            'q1': 'Qualifying 1',
            'q2': 'Qualifying 2',
            'q3': 'Qualifying 3',
            'sprint': 'Sprint',
            'sprint race': 'Sprint',
            'sprint qualifying': 'Sprint Qualifying',
            'sprint shootout': 'Sprint Qualifying',
            'race': 'Race',
            'grand prix': 'Race',
            'gp': 'Race'
        }
        
        # Team mappings
        self.team_mappings = {
            'red bull': 'Red Bull Racing',
            'red bull racing': 'Red Bull Racing',
            'ferrari': 'Ferrari',
            'mercedes': 'Mercedes',
            'mclaren': 'McLaren',
            'aston martin': 'Aston Martin',
            'alpine': 'Alpine',
            'williams': 'Williams',
            'haas': 'Haas',
            'alfa romeo': 'Alfa Romeo',
            'kick sauber': 'Kick Sauber',
            'rb': 'Red Bull Racing'
        }
        
        # Driver mappings
            self.driver_mappings = {
                # McLaren
                'piastri': 'Oscar PIASTRI',
                'oscar piastri': 'Oscar PIASTRI',
                'oscar': 'Oscar PIASTRI',

                'norris': 'Lando NORRIS',
                'lando norris': 'Lando NORRIS',
                'lando': 'Lando NORRIS',

                # Mercedes
                'russell': 'George RUSSELL',
                'george russell': 'George RUSSELL',
                'george': 'George RUSSELL',

                'antonelli': 'Andrea Kimi ANTONELLI',
                'kimi antonelli': 'Andrea Kimi ANTONELLI',
                'andrea kimi antonelli': 'Andrea Kimi ANTONELLI',
                'kimi': 'Andrea Kimi ANTONELLI',

                # Red Bull
                'verstappen': 'Max VERSTAPPEN',
                'max verstappen': 'Max VERSTAPPEN',
                'max': 'Max VERSTAPPEN',

                'lawson': 'Liam LAWSON',
                'liam lawson': 'Liam LAWSON',
                'liam': 'Liam LAWSON',

                # Ferrari
                'leclerc': 'Charles LECLERC',
                'charles leclerc': 'Charles LECLERC',
                'charles': 'Charles LECLERC',

                'hamilton': 'Lewis HAMILTON',
                'lewis hamilton': 'Lewis HAMILTON',
                'lewis': 'Lewis HAMILTON',

                # Haas
                'ocon': 'Esteban OCON',
                'esteban ocon': 'Esteban OCON',
                'esteban': 'Esteban OCON',

                'bearman': 'Oliver BEARMAN',
                'oliver bearman': 'Oliver BEARMAN',
                'oliver': 'Oliver BEARMAN',

                # Williams
                'albon': 'Alex ALBON',
                'alex albon': 'Alex ALBON',
                'alexander albon': 'Alex ALBON',
                'alex': 'Alex ALBON',

                'sainz': 'Carlos SAINZ',
                'carlos sainz': 'Carlos SAINZ',
                'carlos sainz jr.': 'Carlos SAINZ',
                'carlos': 'Carlos SAINZ',

                # Alpine
                'gasly': 'Pierre GASLY',
                'pierre gasly': 'Pierre GASLY',
                'pierre': 'Pierre GASLY',

                'doohan': 'Jack DOOHAN',
                'jack doohan': 'Jack DOOHAN',
                'jack': 'Jack DOOHAN',

                # Aston Martin
                'stroll': 'Lance STROLL',
                'lance stroll': 'Lance STROLL',
                'lance': 'Lance STROLL',

                'alonso': 'Fernando ALONSO',
                'fernando alonso': 'Fernando ALONSO',
                'fernando': 'Fernando ALONSO',

                # RB (Racing Bulls)
                'hadjar': 'Isack HADJAR',
                'isack hadjar': 'Isack HADJAR',
                'isack': 'Isack HADJAR',

                'tsunoda': 'Yuki TSUNODA',
                'yuki tsunoda': 'Yuki TSUNODA',
                'yuki': 'Yuki TSUNODA',

                # Kick Sauber
                'bortoleto': 'Gabriel BORTOLETO',
                'gabriel bortoleto': 'Gabriel BORTOLETO',
                'gabriel': 'Gabriel BORTOLETO',

                'hulkenberg': 'Nico HULKENBERG',
                'nico hulkenberg': 'Nico HULKENBERG',
                'nico hülkenberg': 'Nico HULKENBERG',
                'nico': 'Nico HULKENBERG',
            }
    
    def check_session_availability(self, meeting_name: str, session_type: str) -> Tuple[bool, str]:
        """
        Check if a specific session has happened by querying the database
        Returns: (has_happened, message)
        """
        try:
            with engine.connect() as conn:
                # First check if the meeting exists
                meeting_query = f"SELECT meeting_key FROM meetings WHERE meeting_name = '{meeting_name}'"
                result = conn.execute(text(meeting_query))
                meeting_row = result.fetchone()
                
                if not meeting_row:
                    return False, f"❌ {meeting_name} not found in the database."
                
                meeting_key = meeting_row[0]
                
                # Check if the session exists for this meeting
                session_query = f"""
                SELECT session_name 
                FROM sessions 
                WHERE meeting_key = {meeting_key} 
                AND session_name ILIKE '%{session_type}%'
                """
                result = conn.execute(text(session_query))
                session_rows = result.fetchall()
                
                if session_rows:
                    return True, f"✅ {session_type} for {meeting_name} has been completed and data is available."
                else:
                    # Get available sessions for this meeting
                    available_query = f"""
                    SELECT DISTINCT session_name 
                    FROM sessions 
                    WHERE meeting_key = {meeting_key}
                    ORDER BY session_name
                    """
                    result = conn.execute(text(available_query))
                    available_sessions = [row[0] for row in result.fetchall()]
                    
                    if available_sessions:
                        available_list = ', '.join(available_sessions)
                        
                        # Special handling for Sprint Qualifying
                        if session_type == 'Sprint Qualifying':
                            # Check if Sprint race is available
                            sprint_available = any('sprint' in s.lower() and 'race' in s.lower() for s in available_sessions)
                            if sprint_available:
                                return False, f"❌ Sprint Qualifying data is not yet available for {meeting_name}. However, I do have Sprint Race data! Would you like to see the Sprint Race results instead?"
                            else:
                                return False, f"❌ Sprint Qualifying data is not yet available for {meeting_name}. Available sessions: {available_list}"
                        
                        # Special handling for other missing session types
                        elif session_type in ['Sprint', 'Sprint Race']:
                            return False, f"❌ {session_type} was not scheduled for {meeting_name}. This meeting only had: {available_list}"
                        elif session_type in ['Practice 1', 'Practice 2', 'Practice 3']:
                            return False, f"❌ {session_type} was not scheduled for {meeting_name}. Available sessions: {available_list}"
                        elif session_type == 'Qualifying':
                            return False, f"❌ {session_type} was not scheduled for {meeting_name}. Available sessions: {available_list}"
                        else:
                            return False, f"❌ {session_type} for {meeting_name} not found. Available sessions: {available_list}"
                    else:
                        return False, self.suggest_alternative_sessions(meeting_name, session_type)        
        except Exception as e:
            return False, f"❌ Database error checking session availability: {e}"
    

    def suggest_alternative_sessions(self, meeting_name: str, requested_session: str) -> str:
        """Suggest alternative sessions when the requested session is not available"""
        try:
            with engine.connect() as conn:
                # Get meeting key first
                meeting_query = f"SELECT meeting_key FROM meetings WHERE meeting_name = '{meeting_name}'"
                result = conn.execute(text(meeting_query))
                meeting_row = result.fetchone()
                
                if not meeting_row:
                    return f"❌ {meeting_name} not found in the database."
                
                meeting_key = meeting_row[0]
                
                # Get all available sessions for this meeting
                available_query = f"""
                SELECT DISTINCT session_name 
                FROM sessions 
                WHERE meeting_key = {meeting_key}
                ORDER BY session_name
                """
                result = conn.execute(text(available_query))
                available_sessions = [row[0] for row in result.fetchall()]
                
                if not available_sessions:
                    return f"❌ No sessions found for {meeting_name}."
                
                # Create suggestions based on the requested session type
                suggestions = []
                
                if 'sprint' in requested_session.lower():
                    # For Sprint Qualifying requests, suggest Sprint Race
                    sprint_race = [s for s in available_sessions if 'sprint' in s.lower() and 'race' in s.lower()]
                    if sprint_race:
                        suggestions.append(f"�� **Sprint Race**: I have Sprint Race data available! Ask: 'Show me the Sprint Race results for {meeting_name}'")
                    
                    # Also suggest regular qualifying
                    qualifying = [s for s in available_sessions if 'qualifying' in s.lower() and 'sprint' not in s.lower()]
                    if qualifying:
                        suggestions.append(f"🏁 **Grand Prix Qualifying**: Try: 'Show me the qualifying results for {meeting_name}'")
                
                elif 'qualifying' in requested_session.lower():
                    # For qualifying requests, suggest other session types
                    practice_sessions = [s for s in available_sessions if 'practice' in s.lower()]
                    if practice_sessions:
                        suggestions.append(f"🏁 **Practice Sessions**: Try: 'How did [team/driver] perform in Practice 1/2/3 at {meeting_name}'")
                    
                    race = [s for s in available_sessions if 'race' in s.lower() and 'sprint' not in s.lower()]
                    if race:
                        suggestions.append(f"🏁 **Race Results**: Try: 'Who won the {meeting_name}'")
                
                elif 'practice' in requested_session.lower():
                    # For practice requests, suggest qualifying or race
                    qualifying = [s for s in available_sessions if 'qualifying' in s.lower()]
                    if qualifying:
                        suggestions.append(f"🏁 **Qualifying**: Try: 'Show me qualifying results for {meeting_name}'")
                    
                    race = [s for s in available_sessions if 'race' in s.lower() and 'sprint' not in s.lower()]
                    if race:
                        suggestions.append(f"🏁 **Race Results**: Try: 'Who won the {meeting_name}'")
                
                # Always suggest race results as a fallback
                race = [s for s in available_sessions if 'race' in s.lower() and 'sprint' not in s.lower()]
                if race and 'race' not in requested_session.lower():
                    suggestions.append(f"🏁 **Race Results**: Try: 'Who won the {meeting_name}'")
                
                # Format the response
                response = f"❌ {requested_session} data is not yet available for {meeting_name}.\n\n"
                response += "📋 **Available sessions**: " + ", ".join(available_sessions) + "\n\n"
                
                if suggestions:
                    response += "�� **Suggestions**:\n"
                    for suggestion in suggestions:
                        response += f"  {suggestion}\n"
                else:
                    response += f"�� **Try asking about**: " + ", ".join(available_sessions[:3]) + "\n"
                
                return response
                
        except Exception as e:
            return f"❌ Error suggesting alternatives: {e}"
    
    def get_meeting_session_info(self, meeting_name: str) -> str:
        """Get detailed session information for a meeting"""
        try:
            with engine.connect() as conn:
                # Get meeting key first
                meeting_query = f"SELECT meeting_key, date_start FROM meetings WHERE meeting_name = '{meeting_name}'"
                result = conn.execute(text(meeting_query))
                meeting_row = result.fetchone()
                
                if not meeting_row:
                    return f"❌ {meeting_name} not found in the database."
                
                meeting_key, date_start = meeting_row
                
                # Get all sessions for this meeting
                session_query = f"""
                SELECT session_name, date_start, date_end
                FROM sessions 
                WHERE meeting_key = {meeting_key}
                ORDER BY date_start
                """
                result = conn.execute(text(session_query))
                sessions = result.fetchall()
                
                if not sessions:
                    return f"❌ No sessions found for {meeting_name}."
                
                # Format the response
                info = f"🏎️ {meeting_name} (Date: {date_start})\n"
                info += "�� Scheduled Sessions:\n"
                
                for session_name, start_date, end_date in sessions:
                    info += f"  • {session_name}\n"
                
                return info                  
        except Exception as e:
            return f"❌ Error getting meeting info: {e}"
    
    def check_race_availability(self, meeting_name: str) -> Tuple[bool, str]:
        """
        Check if a race has happened and return appropriate message
        Returns: (has_happened, message)
        """
        return self.check_session_availability(meeting_name, 'Race')
    
    def validate_intent(self, intent: F1QueryIntent) -> Tuple[bool, str]:
        """Validate the entire intent including session type"""
        if intent.meeting:
            return self.check_session_availability(intent.meeting, intent.session_type)
        return True, "No meeting specified"
    
    def extract_f1_entities(self, query: str) -> F1QueryIntent:
        """Extract F1-specific entities from natural language query"""
        
        # Normalize query
        query_lower = query.lower()
        #print(f"🔍 DEBUG: Processing query: '{query}'")
        #print(f"🔍 DEBUG: Lowercase: '{query_lower}'")
        
        # Extract entities
        intent = F1QueryIntent(query_type="general")

            # Extract teams with fuzzy matching
        for team_key, team_name in self.team_mappings.items():
            if team_key in query_lower:
                intent.team = team_name
                break
            # Try fuzzy matching for teams
            elif len(team_key) > 3:  # Only for longer team names
                matches = get_close_matches(team_key, [team_key], n=1, cutoff=0.8)
                if matches and matches[0] in query_lower:
                    intent.team = team_name
                    #print(f"�� Fuzzy matched team: '{team_key}' → '{team_name}'")
                    break
        
        # Extract drivers with fuzzy matching
        drivers_found = []
        for driver_key, driver_name in self.driver_mappings.items():
            if driver_key in query_lower:
                drivers_found.append(driver_name)
            # Try fuzzy matching for drivers
            elif len(driver_key) > 3:  # Only for longer names
                matches = get_close_matches(driver_key, [driver_key], n=1, cutoff=0.7)
                if matches and matches[0] in query_lower:
                    drivers_found.append(driver_name)
                    #print(f"🔍 Fuzzy matched driver: '{driver_key}' → '{driver_name}'")
       
        # Remove duplicates and keep order
        unique_drivers = []
        for driver in drivers_found:
            if driver not in unique_drivers:
                unique_drivers.append(driver)
        
        if len(unique_drivers) >= 1:
            intent.driver1 = unique_drivers[0]
        if len(unique_drivers) >= 2:
            intent.driver2 = unique_drivers[1]
        
        # Extract meetings with fuzzy matching
        for race_key, race_name in self.race_keywords.items():
            if race_key in query_lower:
                intent.meeting = race_name
                break
            # Try fuzzy matching for race names
            elif len(race_key) > 4:
                matches = get_close_matches(race_key, [race_key], n=1, cutoff=0.8)
                if matches and matches[0] in query_lower:
                    intent.meeting = race_name
                    #print(f"�� Fuzzy matched race: '{race_key}' → '{race_name}'")
                    break
        
        # # Extract session type with fuzzy matching
        # for session_key, session_name in self.session_keywords.items():
        #     if session_key in query_lower:
        #         intent.session_type = session_name
        #         break
        #     # Try fuzzy matching for session types
        #     elif len(session_key) > 3:
        #         matches = get_close_matches(session_key, [session_key], n=1, cutoff=0.7)
        #         if matches and matches[0] in query_lower:
        #             intent.session_type = session_name
        #             print(f"🔍 Fuzzy matched session: '{session_key}' → '{session_name}'")
        #             break
        
        # Extract session type with IMPROVED sprint handling
        session_matched = False
        for session_key, session_name in self.session_keywords.items():
            if session_key in query_lower:
                intent.session_type = session_name
                session_matched = True
                break
            # Try fuzzy matching for session types
            elif len(session_key) > 3:
                matches = get_close_matches(session_key, [session_key], n=1, cutoff=0.7)
                if matches and matches[0] in query_lower:
                    intent.session_type = session_name
                    session_matched = True
                    #print(f"🔍 Fuzzy matched session: '{session_key}' → '{session_name}'")
                    break
        
        # SPECIAL HANDLING: If user asks for "sprint qualifying" but we only have sprint race
        if "sprint" in query_lower and "qualifying" in query_lower:
            intent.session_type = "Sprint Qualifying"  # This will trigger the special handling in SQL builder
            session_matched = True


        # Extract lap numbers
        lap_match = re.search(r'lap\s+(\d+)', query_lower)
        if lap_match:
            intent.lap_number = int(lap_match.group(1))
        
        comparison_indicators = ['and', 'vs', 'versus', 'against', 'with', 'or']
        has_comparison_word = any(word in query_lower for word in comparison_indicators)

        # Determine query type
        if 'strategy' in query_lower and intent.team:
            intent.query_type = "strategy"
            #print(f"🔍 DEBUG: Set to strategy")
        elif 'slow' in query_lower or 'incident' in query_lower or 'happened' in query_lower:
            intent.query_type = "incident"
            #print(f"🔍 DEBUG: Set to incident")
        elif ('compare' in query_lower or has_comparison_word) and len(unique_drivers) >= 2:
            intent.query_type = "comparison"
            #print(f"🔍 DEBUG: Set to comparison (detected {len(unique_drivers)} drivers with comparison indicators)")
        elif 'tire' in query_lower or 'tyre' in query_lower:
            intent.query_type = "tire_strategy"
            #print(f"�� DEBUG: Set to tire_strategy")
        elif any(word in query_lower for word in ["won", "winner", "who won", "who got", "pole position", "pole"]) and intent.meeting:
            intent.query_type = "race_results"
            #print(f"�� DEBUG: Set to race_results")
        elif 'qualifying' in query_lower and intent.meeting:
            intent.query_type = "qualifying_results"
            #print(f"🔍 DEBUG: Set to qualifying_results")
        else:
            intent.query_type = "general"
            #print(f"🔍 DEBUG: Set to general (default)")
        
        #print(f"�� DEBUG: Final query_type: {intent.query_type}")
        return intent
    
    def get_available_sessions(self, meeting_name: str = None) -> Dict[str, list[str]]:
        """Get list of available sessions by querying the database"""
        try:
            with engine.connect() as conn:
                if meeting_name:
                    # Get meeting key first
                    meeting_query = f"SELECT meeting_key FROM meetings WHERE meeting_name = '{meeting_name}'"
                    result = conn.execute(text(meeting_query))
                    meeting_row = result.fetchone()
                    
                    if not meeting_row:
                        return {meeting_name: []}
                    
                    meeting_key = meeting_row[0]
                    
                    # Get sessions for this meeting
                    session_query = f"""
                    SELECT DISTINCT session_name 
                    FROM sessions 
                    WHERE meeting_key = {meeting_key}
                    ORDER BY session_name
                    """
                    result = conn.execute(text(session_query))
                    sessions = [row[0] for row in result.fetchall()]
                    return {meeting_name: sessions}
                else:
                    # Get all meetings and their sessions
                    query = """
                    SELECT m.meeting_name, s.session_name
                    FROM meetings m
                    JOIN sessions s ON m.meeting_key = s.meeting_key
                    ORDER BY m.meeting_name, s.session_name
                    """
                    result = conn.execute(text(query))
                    rows = result.fetchall()
                    
                    meetings = {}
                    for meeting, session in rows:
                        if meeting not in meetings:
                            meetings[meeting] = []
                        meetings[meeting].append(session)
                    
                    return meetings
                    
        except Exception as e:
            print(f"Error getting available sessions: {e}")
            return {}
    
    def get_available_races(self) -> list[str]:
        """Get list of available races by querying the database"""
        try:
            with engine.connect() as conn:
                query = """
                SELECT DISTINCT m.meeting_name
                FROM meetings m
                JOIN sessions s ON m.meeting_key = s.meeting_key
                WHERE s.session_name ILIKE '%Race%'
                ORDER BY m.meeting_name
                """
                result = conn.execute(text(query))
                races = [row[0] for row in result.fetchall()]
                return races
        except Exception as e:
            print(f"Error getting available races: {e}")
            return []
    
    def get_all_race_keywords(self) -> dict[str, str]:
        """Get all race keywords for reference"""
        return self.race_keywords.copy()
    
    def get_all_session_keywords(self) -> dict[str, str]:
        """Get all session keywords for reference"""
        return self.session_keywords.copy()

# Initialize the validator
validator = F1QueryValidator()
print("✅ F1QueryValidator initialized successfully!")

✅ F1QueryValidator initialized successfully!


# Test Validator & Intent Identification

In [8]:
# # Test the improved parsing with validator
# test_queries = [
#     # Race queries
#     "What strategy did Red Bull use at Monaco?",
#     "Who won the Monaco race?",
#     "Compare lap times between Verstappen and Hamilton in the Miami race",

    
#     # Practice queries
#     "How did Ferrari perform in Practice 1 at the British Grand Prix?",
#     "What were the fastest lap times in FP2 at Monaco?",
#     "Compare Verstappen and Leclerc in Practice 3 at the Spanish Grand Prix",
    
#     # Qualifying queries
#     "Who got pole position in qualifying at the Australian Grand Prix?",
#     "What happened to Norris in Q2 at the Canadian Grand Prix?",
#     "Compare the qualifying performance of Mercedes and Ferrari at Monaco",
    
#     # Sprint queries
#     "Who won the sprint race at the Austrian Grand Prix?",
#     "What was the sprint qualifying result at the Chinese Grand Prix?",
#     "How did Red Bull perform in the sprint at Miami?",
    
#     # Future/unknown sessions
#     "What strategy will Red Bull use in Practice 1 at the Hungarian Grand Prix?",
#     "How will Ferrari perform in qualifying at the Dutch Grand Prix?",
#     "Compare Verstappen and Hamilton in the sprint at the Italian Grand Prix"
# ]

# print("🧪 Testing Improved Query Parsing with Validator:")
# for query in test_queries:
#     intent = validator.extract_f1_entities(query)
#     is_valid, message = validator.validate_intent(intent)
#     print(f"\nQuery: {query}")
#     print(f"Intent: {intent}")
#     print(f"Validation: {message}")

# # Show available races
# print(f"\n�� Available races: {validator.get_available_races()}")

# Helper Functions to get meeting key, driver number, format lap times

In [9]:
def get_meeting_key(meeting_name: str) -> Optional[int]:
    """Get meeting key from meeting name"""
    try:
        with engine.connect() as conn:
            # Try exact match first
            query = f"SELECT meeting_key FROM meetings WHERE meeting_name = '{meeting_name}'"
            result = conn.execute(text(query))
            row = result.fetchone()
            
            if row:
                return row[0]
            
            # Try partial match
            query = f"SELECT meeting_key, meeting_name FROM meetings WHERE meeting_name ILIKE '%{meeting_name}%'"
            result = conn.execute(text(query))
            rows = result.fetchall()
            
            if rows:
                #print(f"Found meetings: {rows}")
                return rows[0][0]  # Return first match
            
            return None
    except Exception as e:
        print(f"Error getting meeting key: {e}")
        return None

def get_driver_number(driver_name: str, meeting_key: int) -> Optional[int]:
    """Get driver number from driver name and meeting"""
    try:
        with engine.connect() as conn:
            query = f"""
            SELECT DISTINCT driver_number 
            FROM drivers_transformed 
            WHERE full_name = '{driver_name}' 
            AND meeting_key = {meeting_key}
            """
            result = conn.execute(text(query))
            row = result.fetchone()
            return row[0] if row else None
    except Exception as e:
        print(f"Error getting driver number: {e}")
        return None

# # Test the functions work
# print("🧪 Testing Helper Functions:")
# test_meeting = "Australian Grand Prix"
# meeting_key = get_meeting_key(test_meeting)
# print(f"Meeting key for '{test_meeting}': {meeting_key}")

# if meeting_key:
#     driver_num = get_driver_number("Max Verstappen", meeting_key)
#     print(f"Driver number for Max Verstappen: {driver_num}")

In [10]:
def format_lap_time(seconds: float) -> str:
    """Convert seconds to MM:SS.ss format for lap times"""
    if seconds is None or seconds <= 0:
        return "N/A"
    
    minutes = int(seconds // 60)
    remaining_seconds = seconds % 60
    
    return f"{minutes:02d}:{remaining_seconds:06.3f}"[:-3]  # Remove microseconds, keep 2 decimal places

def detect_unsupported_features(query: str) -> tuple[bool, str]:
    """Detect if the query requests unsupported features and return appropriate response"""
    query_lower = query.lower()
    
    # Features we can't support yet
    unsupported_patterns = [
        # Visualization requests
        ('graph', 'chart', 'plot', 'visualization', 'visualize', 'show me a graph', 'create a chart'),
        ('image', 'picture', 'photo', 'screenshot', 'diagram'),
        ('bar chart', 'line graph', 'pie chart', 'scatter plot'),
        
        # Advanced analysis requests
        ('predict', 'forecast', 'future', 'next race', 'will win'),
        ('simulate', 'simulation', 'what if'),
        ('trend', 'pattern analysis', 'statistical analysis'),
        
        # Export requests
        ('export', 'download', 'save as', 'csv', 'excel', 'pdf'),
        
        # Real-time requests
        ('live', 'real time', 'current', 'now', 'today'),
        
        # Complex calculations
        ('calculate', 'compute', 'formula', 'equation'),
        
        # Media requests
        ('video', 'audio', 'stream', 'broadcast'),
        
        # Interactive features
        ('interactive', 'click', 'hover', 'zoom', 'filter')
    ]
    
    for pattern_group in unsupported_patterns:
        for pattern in pattern_group:
            if pattern in query_lower:
                # Generate appropriate response based on the type of request
                if any(word in query_lower for word in ['graph', 'chart', 'plot', 'visualization']):
                    return True, "📊 I can't generate graphs or charts just yet, but keep a look out for future updates! For now, I can provide detailed text-based analysis of the data."
                elif any(word in query_lower for word in ['image', 'picture', 'photo']):
                    return True, "🖼️ I can't generate images or pictures just yet, but keep a look out for future updates! I can provide detailed text-based analysis instead."
                elif any(word in query_lower for word in ['predict', 'forecast', 'future']):
                    return True, "🔮 I can't make predictions about future races just yet, but keep a look out for future updates! I can analyze historical data from completed races."
                elif any(word in query_lower for word in ['export', 'download', 'save']):
                    return True, "💾 I can't export data just yet, but keep a look out for future updates! I can provide detailed analysis of the data in our conversation."
                elif any(word in query_lower for word in ['live', 'real time', 'current']):
                    return True, "⏰ I can only analyze historical data from completed races. I can't provide real-time information just yet, but keep a look out for future updates!"
                else:
                    return True, "🤖 I can't do that just yet, but keep a look out for future updates! I can help you analyze F1 race data with detailed text-based insights."
    
    return False, ""

# Get DB Schema for model

In [11]:
def generate_schema_description():
    """Generate a detailed schema description for the model"""
    
    # Define the tables we want to include
    target_tables = [
        'sessions_transformed',
        'drivers_transformed', 
        'intervals_transformed',
        'laps_transformed',
        'pit_stops_transformed',
        'positions_transformed',
        'stints_transformed',
        'weather_transformed',
        'race_control',
        'meetings'
    ]
    
    schema_description = """
    # F1 Database Schema and Data Format
    
    ## Table Descriptions and Sample Data:
    
    """
    
    try:
        with engine.connect() as conn:
            for table in target_tables:
                schema_description += f"\n### {table}\n"
                
                # Check if table exists
                table_exists_query = f"""
                SELECT EXISTS (
                    SELECT FROM information_schema.tables 
                    WHERE table_schema = 'public' 
                    AND table_name = '{table}'
                );
                """
                result = conn.execute(text(table_exists_query))
                table_exists = result.fetchone()[0]
                
                if not table_exists:
                    schema_description += f"**Status:** Table does not exist in database\n\n"
                    continue
                
                # Get column information
                columns_query = f"""
                SELECT column_name, data_type, is_nullable
                FROM information_schema.columns 
                WHERE table_name = '{table}'
                ORDER BY ordinal_position
                """
                result = conn.execute(text(columns_query))
                columns = result.fetchall()
                
                schema_description += "**Columns:**\n"
                for col_name, data_type, nullable in columns:
                    schema_description += f"- `{col_name}`: {data_type} ({'NULL' if nullable == 'YES' else 'NOT NULL'})\n"
                
                # Get sample data
                sample_query = f"SELECT * FROM {table} LIMIT 5"
                try:
                    result = conn.execute(text(sample_query))
                    rows = result.fetchall()
                    column_names = list(result.keys())
                    
                    if rows:
                        schema_description += f"\n**Sample Data ({len(rows)} rows):**\n"
                        for i, row in enumerate(rows):
                            row_dict = dict(zip(column_names, row))
                            schema_description += f"Row {i+1}: {row_dict}\n"
                    else:
                        schema_description += "\n**Sample Data:** No rows found\n"
                        
                except Exception as e:
                    schema_description += f"\n**Sample Data:** Error - {e}\n"
                
                schema_description += "\n"
                
    except Exception as e:
        schema_description += f"\n❌ Error generating schema: {e}\n"
    
    return schema_description

# Generate the schema description
schema_description = generate_schema_description()
print("📋 Generated Schema Description:")
print(schema_description)

📋 Generated Schema Description:

    # F1 Database Schema and Data Format

    ## Table Descriptions and Sample Data:

    
### sessions_transformed
**Columns:**
- `session_key`: integer (NULL)
- `meeting_key`: integer (NULL)
- `session_name`: character varying (NULL)
- `session_type`: character varying (NULL)
- `date_start`: timestamp without time zone (NULL)
- `date_end`: timestamp without time zone (NULL)
- `created_at`: timestamp without time zone (NULL)
- `session_type_encoded`: integer (NULL)

**Sample Data (5 rows):**
Row 1: {'session_key': 9686, 'meeting_key': 1254, 'session_name': 'Practice 1', 'session_type': 'Practice', 'date_start': datetime.datetime(2025, 3, 14, 1, 30), 'date_end': datetime.datetime(2025, 3, 14, 2, 30), 'created_at': datetime.datetime(2025, 7, 8, 11, 56, 35, 493407), 'session_type_encoded': 0}
Row 2: {'session_key': 9687, 'meeting_key': 1254, 'session_name': 'Practice 2', 'session_type': 'Practice', 'date_start': datetime.datetime(2025, 3, 14, 5, 0), 'date

# -----------------------BUILDING SQL QUERIES & ANALYSIS PROMPTS-----------------------------

In [12]:
def build_f1_sql_query(intent: F1QueryIntent, validator: F1QueryValidator, db_schema: str = "", query: str = "", clarified_session: str = None) -> str:
    """Build dynamic SQL queries based on query type and complexity"""
    
    # Validate the intent first
    is_valid, message = validator.validate_intent(intent)
    if not is_valid:
        return f"-- {message}"
    
    # Validate intent before building queries
    if not intent.meeting:
        return "-- No meeting specified in the query"
    
    if intent.query_type == "comparison" and not intent.driver1 and not intent.driver2:
        return "-- Please specify which drivers you want to compare (e.g., 'Compare Verstappen and Hamilton')"
    
    # Get meeting key using the helper function
    meeting_key = get_meeting_key(intent.meeting)
    
    if not meeting_key:
        return f"-- No meeting found for: {intent.meeting}"
    
    session_key = None
    available_sessions = []
    
    try:
        with engine.connect() as conn:
            # First, get all available sessions for this meeting
            sessions_query = f"""
            SELECT session_name, session_type 
            FROM sessions_transformed 
            WHERE meeting_key = {meeting_key}
            ORDER BY session_name
            """
            result = conn.execute(text(sessions_query))
            available_sessions = [row[0] for row in result.fetchall()]
            
            # If we have a clarified session, use it directly
            if clarified_session:
                session_query = f"""
                SELECT session_key 
                FROM sessions_transformed 
                WHERE meeting_key = {meeting_key} 
                AND session_name = '{clarified_session}'
                """
                result = conn.execute(text(session_query))
                session_row = result.fetchone()
                if session_row:
                    session_key = session_row[0]
                else:
                    return f"-- Session '{clarified_session}' not found for {intent.meeting}"
            
            # Check if the requested session exists - FIXED FOR QUALIFYING
            elif intent.query_type == "qualifying_results" or "qualifying" in intent.session_type.lower():
                # Special handling for qualifying sessions
                qualifying_query = f"""
                SELECT DISTINCT session_name, session_type 
                FROM sessions_transformed 
                WHERE meeting_key = {meeting_key}
                AND session_name LIKE '%Qualifying%'
                ORDER BY session_name
                """
                result = conn.execute(text(qualifying_query))
                qualifying_sessions = result.fetchall()
                
                # Check for Sprint Race sessions (since we don't have Sprint Qualifying)
                sprint_query = f"""
                SELECT DISTINCT session_name, session_type 
                FROM sessions_transformed 
                WHERE meeting_key = {meeting_key}
                AND session_name LIKE '%Sprint%'
                ORDER BY session_name
                """
                result = conn.execute(text(sprint_query))
                sprint_sessions = result.fetchall()
                
                # Handle Sprint Qualifying requests specially
                if "sprint" in intent.session_type.lower() and "qualifying" in intent.session_type.lower():
                    if sprint_sessions:
                        # We have Sprint Race data, but user wants Sprint Qualifying
                        sprint_race_name = sprint_sessions[0][0]
                        return f"-- Sprint Qualifying data is not available for {intent.meeting}. However, I do have Sprint Race data ('{sprint_race_name}'). Would you like to see the Sprint Race results instead? Just ask: 'Show me the Sprint Race results for {intent.meeting}'"
                    else:
                        return f"-- No Sprint sessions found for {intent.meeting}. Available sessions: {', '.join(available_sessions)}"
                
                # For general "qualifying" requests, check what we have
                all_available_options = []
                if qualifying_sessions:
                    all_available_options.extend([s[0] for s in qualifying_sessions])
                if sprint_sessions:
                    all_available_options.extend([s[0] for s in sprint_sessions])
                
                # If user asked for generic "qualifying" and we have both regular qualifying and sprint
                if (len(all_available_options) > 1 and 
                    intent.session_type in ['Qualifying', 'qualifying'] and
                    not ("sprint" in intent.session_type.lower())):
                    
                    # Ask for clarification but explain what each option is
                    clarification_message = f"Multiple sessions found for {intent.meeting}:\n"
                    if qualifying_sessions:
                        clarification_message += f"- {qualifying_sessions[0][0]} (Grand Prix Qualifying)\n"
                    if sprint_sessions:
                        clarification_message += f"- {sprint_sessions[0][0]} (Sprint Race - no Sprint Qualifying available)\n"
                    clarification_message += "Which would you like to see?"
                    
                    return {
                        "type": "clarification_needed",
                        "message": clarification_message,
                        "options": all_available_options,
                        "meeting_key": meeting_key,
                        "meeting_name": intent.meeting
                    }
                
                # Try to find exact match first
                exact_match = None
                all_sessions = qualifying_sessions + sprint_sessions
                for session_name, session_type in all_sessions:
                    if intent.session_type.lower() in session_name.lower():
                        exact_match = session_name
                        break
                
                if exact_match:
                    # Get session key for exact match
                    session_query = f"""
                    SELECT session_key 
                    FROM sessions_transformed 
                    WHERE meeting_key = {meeting_key} 
                    AND session_name = '{exact_match}'
                    """
                    result = conn.execute(text(session_query))
                    session_row = result.fetchone()
                    if session_row:
                        session_key = session_row[0]
                else:
                    # No exact match, use first qualifying session as default
                    if qualifying_sessions:
                        default_session = qualifying_sessions[0][0]
                        session_query = f"""
                        SELECT session_key 
                        FROM sessions_transformed 
                        WHERE meeting_key = {meeting_key} 
                        AND session_name = '{default_session}'
                        """
                        result = conn.execute(text(session_query))
                        session_row = result.fetchone()
                        if session_row:
                            session_key = session_row[0]
            else:
                # Regular session handling (non-qualifying)
                session_query = f"""
                SELECT session_key 
                FROM sessions_transformed 
                WHERE meeting_key = {meeting_key} 
                AND UPPER(session_name) LIKE UPPER('%{intent.session_type}%')
                """
                result = conn.execute(text(session_query))
                session_row = result.fetchone()
                if session_row:
                    session_key = session_row[0]
                else:
                    # Handle specific session type issues
                    if "sprint" in intent.session_type.lower():
                        return f"-- Sprint races are not scheduled for all Grand Prix events. The {intent.meeting} only had: {', '.join(available_sessions)}. Would you like to know about the race results instead?"
                    
                    elif "qualifying" in intent.session_type.lower() and any("q" in s.lower() for s in [intent.session_type]):
                        return f"-- Currently I can only analyze overall qualifying sessions, not specific Q1/Q2/Q3 segments. The {intent.meeting} qualifying session included: {', '.join([s for s in available_sessions if 'qualifying' in s.lower()])}. Would you like to see the overall qualifying results?"
                    
                    else:
                        return f"-- The {intent.session_type} session was not held at the {intent.meeting}. Available sessions were: {', '.join(available_sessions)}. What would you like to know about instead?"
                        
    except Exception as e:
        return f"-- Error finding session: {e}"
    
    # If we still don't have a session key, return error
    if not session_key:
        return f"-- No valid session found for {intent.session_type} at {intent.meeting}"
    
    # Dynamic query generation based on query type
    queries = {}
    
    # 🟥 STRATEGY QUERIES
    if intent.query_type == "strategy":
        team_condition = f"UPPER(d.team_name) = UPPER('{intent.team}')" if intent.team else "1=1"  # FIXED: removed extra )
        driver_condition = f"UPPER(d.full_name) = UPPER('{intent.driver1}')" if intent.driver1 else "1=1"
        
        queries["pit_stops"] = f"""
            SELECT 
                COUNT(*) AS total_stops,
                ROUND(AVG(ps.pit_duration)::numeric, 2) AS avg_pit_time,
                MIN(ps.lap_number) AS first_stop,
                MAX(ps.lap_number) AS last_stop,
                COUNT(CASE WHEN ps.long_pit_stop = true THEN 1 END) AS long_stops
            FROM pit_stops_transformed ps
            JOIN drivers_transformed d ON ps.driver_number = d.driver_number 
                AND ps.meeting_key = d.meeting_key
                AND ps.session_key = d.session_key
            WHERE ({team_condition} AND {driver_condition})
              AND ps.meeting_key = {meeting_key}
              AND ps.session_key = {session_key};
        """
        
        queries["stints"] = f"""
            SELECT 
                s.compound,
                COUNT(*) AS stint_count,
                ROUND(AVG(s.stint_duration)::numeric, 2) AS avg_stint_duration,
                MIN(s.lap_start) AS earliest_start,
                MAX(s.lap_end) AS latest_end
            FROM stints_transformed s
            JOIN drivers_transformed d ON s.driver_number = d.driver_number 
                AND s.meeting_key = d.meeting_key
                AND s.session_key = d.session_key
            WHERE ({team_condition} AND {driver_condition})
              AND s.meeting_key = {meeting_key}
              AND s.session_key = {session_key}
            GROUP BY s.compound
            ORDER BY stint_count DESC;
        """
        
        queries["session_comparison"] = f"""
            SELECT 
                ROUND(AVG(ps.pit_duration)::numeric, 2) AS session_avg_pit,
                MIN(ps.pit_duration) AS session_fastest_pit,
                COUNT(*) AS total_session_stops
            FROM pit_stops_transformed ps
            WHERE ps.meeting_key = {meeting_key}
              AND ps.session_key = {session_key};
        """
    
    # 🟦 INCIDENT QUERIES
    elif intent.query_type == "incident" and intent.driver1 and intent.lap_number:
        queries["incident_analysis"] = f"""
            SELECT 
                l.lap_number,
                l.lap_duration,
                l.duration_sector_1,
                l.duration_sector_2,
                l.duration_sector_3,
                l.had_incident,
                l.safety_car_lap,
                l.is_outlier,
                d.full_name,
                d.team_name
            FROM laps_transformed l
            JOIN drivers_transformed d ON l.driver_number = d.driver_number AND l.meeting_key = d.meeting_key
            WHERE UPPER(d.full_name) = UPPER('{intent.driver1}')
              AND l.meeting_key = {meeting_key}
              AND l.session_key = {session_key}
              AND l.lap_number BETWEEN {intent.lap_number - 3} AND {intent.lap_number + 3}
            ORDER BY l.lap_number;
        """
        
        queries["session_context"] = f"""
            SELECT 
                ROUND(AVG(l.lap_duration)::numeric, 3) AS session_avg_lap,
                MIN(l.lap_duration) AS session_best_lap,
                COUNT(CASE WHEN l.had_incident = true THEN 1 END) AS total_incidents
            FROM laps_transformed l
            WHERE l.meeting_key = {meeting_key}
              AND l.session_key = {session_key};
        """
    
    # 🟣 COMPARISON QUERIES
    elif intent.query_type == "comparison" and intent.driver1 and intent.driver2:
        queries["driver_comparison"] = f"""
            SELECT 
                d.full_name,
                d.team_name,
                COUNT(DISTINCT l.lap_number) AS total_laps,
                ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                MIN(l.lap_duration) AS best_lap,
                MAX(l.lap_duration) AS worst_lap,
                ROUND(STDDEV(l.lap_duration)::numeric, 3) AS consistency,
                COUNT(CASE WHEN l.had_incident = true THEN 1 END) AS incidents
            FROM laps_transformed l
            JOIN drivers_transformed d 
                ON l.driver_number = d.driver_number 
                AND l.meeting_key = d.meeting_key
                AND l.session_key = d.session_key
            WHERE 
                (UPPER(d.full_name) = UPPER('{intent.driver1}') OR UPPER(d.full_name) = UPPER('{intent.driver2}'))
                AND l.meeting_key = {meeting_key}
                AND l.session_key = {session_key}
                AND l.lap_duration IS NOT NULL
                AND l.lap_duration > 0
                AND COALESCE(l.is_outlier, false) = false
            GROUP BY d.full_name, d.team_name
            ORDER BY avg_lap ASC;
        """
        
        queries["pit_stop_comparison"] = f"""
            SELECT 
                d.full_name,
                COUNT(*) AS num_stops,
                ROUND(AVG(ps.pit_duration)::numeric, 2) AS avg_stop_time,
                MIN(ps.pit_duration) AS fastest_stop
            FROM pit_stops_transformed ps
            JOIN drivers_transformed d ON ps.driver_number = d.driver_number 
                AND ps.meeting_key = d.meeting_key
                AND ps.session_key = d.session_key
            WHERE (UPPER(d.full_name) = UPPER('{intent.driver1}') OR UPPER(d.full_name) = UPPER('{intent.driver2}'))
              AND ps.meeting_key = {meeting_key}
              AND ps.session_key = {session_key}
            GROUP BY d.full_name
            ORDER BY avg_stop_time;
        """

    # 🟨 GENERAL QUERIES - Use positions_transformed for actual race results
    elif intent.query_type == "general":
        
        team_condition = f"UPPER(d.team_name) = UPPER('{intent.team}')" if intent.team else "1=1"
        driver_condition = f"UPPER(d.full_name) = UPPER('{intent.driver1}')" if intent.driver1 else "1=1"
        
        # Determine session filter based on user intent
        session_filter = ""
        if intent.session_type and intent.session_type != "Race":
            # If user specified a specific session (Practice, Qualifying, etc.)
            session_filter = f"AND s.session_type = '{intent.session_type}'"
        elif "weekend" in query.lower() or "all sessions" in query.lower():
            # If user asks about the whole weekend, don't filter by session
            session_filter = ""
        else:
            # Default: assume they want the race unless they specified otherwise
            session_filter = "AND s.session_type = 'Race'"
        
        # Check if this is a race session (to determine if we should use positions or laps)
        session_type_query = f"""
        SELECT session_type FROM sessions_transformed 
        WHERE session_key = {session_key}
        """
        
        try:
            with engine.connect() as conn:
                result = conn.execute(text(session_type_query))
                session_type = result.fetchone()[0] if result.fetchone() else "Unknown"
        except:
            session_type = "Unknown"
        
        # For race sessions, use positions_transformed to get actual finishing order
        if "race" in session_type.lower():
            queries["race_results"] = f"""
                SELECT 
                    d.full_name,
                    d.team_name,
                    p.position AS finish_position,
                    MIN(l.lap_duration) AS best_lap,
                    ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                    COUNT(l.lap_number) AS total_laps
                FROM positions_transformed p
                JOIN drivers_transformed d ON p.driver_number = d.driver_number AND p.meeting_key = d.meeting_key
                LEFT JOIN laps_transformed l ON p.driver_number = l.driver_number 
                    AND p.meeting_key = l.meeting_key 
                    AND p.session_key = l.session_key
                WHERE ({team_condition} AND {driver_condition})
                AND p.meeting_key = {meeting_key}
                AND p.session_key = {session_key}
                GROUP BY d.full_name, d.team_name, p.position
                ORDER BY p.position ASC;
            """
        else:
            # For non-race sessions (practice, qualifying), use laps_transformed with session filtering
            queries["performance_summary"] = f"""
                SELECT 
                    d.full_name,
                    d.team_name,
                    s.session_name,
                    COUNT(DISTINCT l.lap_number) AS total_laps,
                    ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                    MIN(l.lap_duration) AS best_lap,
                    ROUND(STDDEV(l.lap_duration)::numeric, 3) AS consistency,
                    ROW_NUMBER() OVER (ORDER BY MIN(l.lap_duration)) AS rank_by_best_lap
                FROM laps_transformed l
                JOIN drivers_transformed d 
                    ON l.driver_number = d.driver_number 
                    AND l.meeting_key = d.meeting_key 
                    AND l.session_key = d.session_key
                JOIN sessions_transformed s 
                    ON l.session_key = s.session_key 
                    AND l.meeting_key = s.meeting_key
                WHERE 
                    ({team_condition} AND {driver_condition})
                    AND l.meeting_key = {meeting_key}
                    AND l.session_key = {session_key}
                    AND l.lap_duration IS NOT NULL
                    AND l.lap_duration > 0
                    AND COALESCE(l.is_outlier, false) = false
                GROUP BY 
                    d.full_name, d.team_name, s.session_name
                ORDER BY 
                    MIN(l.lap_duration) ASC;
            """
    
    # 🟧 RACE RESULTS QUERIES - Use positions_transformed for actual race results
    elif intent.query_type == "race_results" or ("won" in intent.query_type.lower() and "race" in intent.query_type.lower()):
        queries["race_results"] = f"""
            WITH final_positions AS (
                SELECT *,
                    ROW_NUMBER() OVER (PARTITION BY driver_number ORDER BY date DESC) AS rn
                FROM positions_transformed
                WHERE meeting_key = {meeting_key} AND session_key = {session_key}
            ),
            driver_info AS (
                SELECT *,
                    ROW_NUMBER() OVER (PARTITION BY driver_number, meeting_key ORDER BY id) AS rn
                FROM drivers_transformed
                WHERE meeting_key = {meeting_key}
            )
            SELECT 
                d.full_name,
                d.team_name,
                p.position AS finish_position
            FROM final_positions p
            JOIN driver_info d 
                ON p.driver_number = d.driver_number AND p.meeting_key = d.meeting_key
            WHERE p.rn = 1
            AND d.rn = 1
            AND p.position IS NOT NULL
            ORDER BY p.position ASC
        """
    
    # 🟪 QUALIFYING RESULTS QUERIES - Use laps_transformed for qualifying
    elif intent.query_type == "qualifying_results" or "pole" in intent.query_type.lower():
        # Add driver/team filtering
        team_condition = f"UPPER(d.team_name) = UPPER('{intent.team}')" if intent.team else "1=1"
        driver_condition = f"UPPER(d.full_name) = UPPER('{intent.driver1}')" if intent.driver1 else "1=1"
        
        # Check if user wants specific driver/team or general results
        if intent.driver1 or intent.team:
            # Specific driver/team query
            queries["qualifying_results"] = f"""
                SELECT 
                    d.full_name,
                    d.team_name,
                    s.session_name,
                    MIN(l.lap_duration) AS best_lap,
                    ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                    COUNT(DISTINCT l.lap_number) AS total_laps,
                    ROW_NUMBER() OVER (ORDER BY MIN(l.lap_duration)) AS qualifying_position
                FROM laps_transformed l
                JOIN drivers_transformed d 
                    ON l.driver_number = d.driver_number AND l.meeting_key = d.meeting_key
                JOIN sessions_transformed s 
                    ON l.session_key = s.session_key AND l.meeting_key = s.meeting_key
                WHERE ({team_condition} AND {driver_condition})
                AND l.meeting_key = {meeting_key}
                AND l.session_key = {session_key}
                GROUP BY d.full_name, d.team_name, s.session_name
                ORDER BY best_lap ASC;
            """
        else:
            # General qualifying results (top 10)
            queries["qualifying_results"] = f"""
                SELECT 
                    d.full_name,
                    d.team_name,
                    s.session_name,
                    MIN(l.lap_duration) AS best_lap,
                    ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                    COUNT(DISTINCT l.lap_number) AS total_laps,
                    ROW_NUMBER() OVER (ORDER BY MIN(l.lap_duration)) AS qualifying_position
                FROM laps_transformed l
                JOIN drivers_transformed d 
                    ON l.driver_number = d.driver_number AND l.meeting_key = d.meeting_key
                JOIN sessions_transformed s 
                    ON l.session_key = s.session_key AND l.meeting_key = s.meeting_key
                WHERE l.meeting_key = {meeting_key}
                AND l.session_key = {session_key}
                GROUP BY d.full_name, d.team_name, s.session_name
                ORDER BY best_lap ASC
                LIMIT 10;
            """
        
    # Fallback for unknown query types
    if not queries:
        queries["basic_summary"] = f"""
            SELECT 
                d.full_name,
                d.team_name,
                COUNT(DISTINCT l.lap_number) AS total_laps,
                ROUND(AVG(l.lap_duration)::numeric, 3) AS avg_lap,
                MIN(l.lap_duration) AS best_lap
            FROM laps_transformed l
            JOIN drivers_transformed d 
                ON l.driver_number = d.driver_number
                AND l.meeting_key = d.meeting_key
                AND l.session_key = d.session_key
            WHERE 
                l.meeting_key = {meeting_key}
                AND l.session_key = {session_key}
                AND l.lap_duration IS NOT NULL
                AND l.lap_duration > 0
                AND COALESCE(l.is_outlier, false) = false
            GROUP BY d.full_name, d.team_name
            ORDER BY best_lap ASC;
        """
    
    # Combine all queries
    sql_parts = []
    for query_name, query_sql in queries.items():
        sql_parts.append(f"-- {query_name.upper()} ANALYSIS\n\n{query_sql}")
    
    return "\n\n".join(sql_parts)

def get_analysis_prompt(intent: F1QueryIntent, query: str, data_summary: str, conversation_context: dict = None, is_follow_up: bool = False) -> str:
    """Generate dynamic analysis prompts based on query type with conversation context"""
    
    base_prompt = f"""
    You are an expert F1 analyst. Analyze the following query and provide a detailed, data-driven response.

    QUERY: {query}
    MEETING: {intent.meeting}
    INTENT: {intent}
    """
    
    # Add context awareness for follow-up questions
    if is_follow_up and conversation_context and conversation_context.get('last_question'):
        last_intent = conversation_context.get('last_intent')
        base_prompt += f"""
    
    CONTEXT: This is a follow-up question to: "{conversation_context['last_question']}"
    Previous query type: {last_intent.query_type if last_intent else 'unknown'}
    Previous meeting: {last_intent.meeting if last_intent else 'unknown'}
    
    The user is asking the same type of question but for a different race/meeting.
    Provide the same type of analysis as before, but for the new meeting.
    """
    
    base_prompt += f"""
    
    ACTUAL DATABASE RESULTS:
    {data_summary}
    
    INSTRUCTIONS:
    1. Look at the ACTUAL data returned from the database queries above
    2. Use the REAL numbers, times, and values from the data
    3. Provide specific analysis based on what the data shows
    4. Do NOT give generic analysis - use the actual data values provided
    5. IMPORTANT: Convert all lap times from seconds to MM:SS.ss format (e.g., 93.0 seconds becomes 01:33.00)
    6. For consistency metrics, keep them in seconds but format lap times properly
    """
   
    # PRIORITY: Check query type FIRST, before winner detection
    if intent.query_type == "comparison":
        return base_prompt + f"""
        
        COMPARISON ANALYSIS FOCUS:
        - Compare the performance of {intent.driver1 or 'the first driver'} and {intent.driver2 or 'the second driver'}
        - Analyze metrics like average lap time, consistency, best/worst laps, incidents
        - Highlight strengths, weaknesses, and who performed better overall
        - Support your analysis with specific numbers and lap times (use MM:SS.ss format)
        - Mention any significant differences in pit stop strategies or tire usage
        - DO NOT say who won the race - this is a comparison analysis, not race results
        - Focus on comparing the two drivers' performance metrics
        """
    
    # STRATEGY ANALYSIS
    elif intent.query_type == "strategy":
        return base_prompt + f"""
        
        STRATEGY ANALYSIS FOCUS:
        - Explain the race strategy used by {intent.team or intent.driver1}
        - Consider number of pit stops, compound usage, stint durations, timing of stops
        - Look for unusual patterns (e.g., long stints, undercuts, early/late stops)
        - Use comparisons when helpful (e.g., against session averages)
        - Mention specific lap numbers, pit stop durations, and tire compounds used
        - Format lap times in MM:SS.ss format
        """
    
    # INCIDENT ANALYSIS
    elif intent.query_type == "incident":
        return base_prompt + f"""
        
        INCIDENT ANALYSIS FOCUS:
        - Analyze what happened to {intent.driver1} around lap {intent.lap_number}
        - Look for sudden changes in lap time, sector times, incidents, or safety car periods
        - Explain the potential reason(s) for their slowdown or issue
        - Consider any contributing factors (weather, track conditions, etc.)
        - Use specific lap times and sector data to support your analysis (format as MM:SS.ss)
        """
    
    # RACE RESULTS ANALYSIS
    elif intent.query_type == "race_results":
        return base_prompt + f"""
        
        RACE RESULTS ANALYSIS FOCUS:
        - Look for the 'finish_position' column - the driver with finish_position = 1 is the WINNER
        - The data is already sorted by finish_position ASC, so the first row will be the winner
        - Give a direct answer: "[Driver Name] won the [Race Name]"
        - Keep it concise and direct
        - DO NOT say "unfortunately" or "no data available" - the data IS there
        - DO NOT look for lap times or other metrics - this is pure race results data

        CRITICAL: The winner is the driver with finish_position = 1, which will be in the first row of the results.
        """
    
    # QUALIFYING RESULTS ANALYSIS
    elif intent.query_type == "qualifying_results":
        return base_prompt + f"""
        
        QUALIFYING RESULTS ANALYSIS FOCUS:
        - Summarize the qualifying session and who secured pole position
        - List the top-performing drivers and their lap times (use MM:SS.ss format)
        - Describe how competitive the session was
        - Highlight close gaps or surprise performances
        - Mention any notable incidents or weather conditions
        """
    
    # GENERAL ANALYSIS
    elif intent.query_type == "general":
        return base_prompt + f"""
        
        GENERAL PERFORMANCE ANALYSIS FOCUS:
        - Summarize the performance of {intent.team or intent.driver1 or 'the session'}
        - Note which session this data represents (Race, Practice, Qualifying, etc.)
        - Comment on lap times, consistency, total laps completed, relative pace
        - Mention standout drivers, tire usage trends, or any notable events
        - Provide context about their position among other drivers or teams
        - For winner/pole questions, give direct answers: "[Driver] won" or "[Driver] got pole"
        - Don't over-analyze simple questions - give direct answers when appropriate
        - Format all lap times in MM:SS.ss format
        
        IMPORTANT: If the data shows session_type, mention which session this analysis covers.
        """
    
    # Default analysis
    return base_prompt + """
    
    GENERAL ANALYSIS FOCUS:
    - Provide a comprehensive analysis of the session data
    - Use specific numbers, times, and statistics from the data
    - Highlight key insights and notable performances
    - Explain any patterns or trends you observe
    - Format lap times in MM:SS.ss format
    """

# -----------INTERACTIVE TESTING WITH CONVERSATIONAL MEMORY---------------

In [15]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import json

# Create conversational memory
memory = ConversationBufferWindowMemory(
    k=5,  # Remember last 5 exchanges
    return_messages=True,
    memory_key="chat_history"
)

# Store conversation context separately
conversation_context = {
    'last_intent': None,
    'last_question': None,
    'conversation_count': 0,
    'ambiguous_session_memory': None
}

def execute_sql_queries(sql_query: str) -> list:
    """Execute SQL queries and return results"""
    query_results = []
    try:
        with engine.connect() as conn:
            # Split the SQL into individual queries
            sql_parts = []
            for line in sql_query.split('\n'):
                line = line.strip()
                if line and not line.startswith('--'):
                    sql_parts.append(line)
            
            # Join and split by semicolons to get individual queries
            combined_sql = ' '.join(sql_parts)
            individual_queries = [q.strip() for q in combined_sql.split(';') if q.strip()]
            
            if not individual_queries:
                return []
            
            # Execute each query separately
            for i, query_sql in enumerate(individual_queries):
                try:
                    result = conn.execute(text(query_sql))
                    rows = result.fetchall()
                    columns = list(result.keys())
                    
                    query_results.append({
                        'query_name': f'Query_{i+1}',
                        'query': query_sql,
                        'columns': columns,
                        'rows': rows
                    })
                    
                except Exception as e:
                    # print(f"  ❌ Error in Query {i+1}: {e}")
                    continue
                    
    except Exception as e:
        # print(f"❌ SQL Execution Error: {e}")
        pass
    
    return query_results

def format_data_summary(query_results: list) -> str:
    """Format query results for the prompt"""
    data_summary = ""
    if not query_results:
        data_summary = "❌ No data returned from database queries."
    else:
        for j, result in enumerate(query_results):
            data_summary += f"\n=== {result.get('query_name', f'QUERY {j+1}')} RESULTS ===\n"
            data_summary += f"Query: {result['query']}\n"
            data_summary += f"Columns: {result['columns']}\n"
            data_summary += f"Rows returned: {len(result['rows'])}\n"
            
            if result['rows']:
                data_summary += "ACTUAL DATA:\n"
                for k, row in enumerate(result['rows']):
                    # Convert row to dict for better formatting
                    row_dict = dict(zip(result['columns'], row))
                    data_summary += f"  Row {k+1}: {row_dict}\n"
            else:
                data_summary += "No data returned for this query.\n"
            data_summary += "\n"
    
    return data_summary

def handle_clarification_response(user_input: str, conversation_context: dict) -> tuple:
    """Handle clarification responses and return (success, intent, original_query)"""
    mem = conversation_context['ambiguous_session_memory']
    clarification = user_input.strip()
    
    # More flexible matching for clarification
    matched = None
    if (clarification.lower() != mem['original_query'].lower() and 
        len(clarification.split()) <= 3):
        
        clarification_lower = clarification.lower()
        for opt in mem['options']:
            opt_lower = opt.lower()
            if (clarification_lower == opt_lower or 
                (clarification_lower == 'sprint' and 'sprint' in opt_lower) or
                (clarification_lower in ['qualifying', 'gp', 'grand prix'] and 'qualifying' in opt_lower and 'sprint' not in opt_lower) or
                (clarification == '1' and opt == mem['options'][0]) or
                (clarification == '2' and len(mem['options']) > 1 and opt == mem['options'][1])):
                matched = opt
                break

    if matched:
        # Extract original intent and override session type
        intent = validator.extract_f1_entities(mem['original_query'])
        intent.session_type = matched
        intent.meeting = mem.get('meeting_name')
        
        # Clear ambiguous memory
        conversation_context['ambiguous_session_memory'] = None
        
        return True, intent, mem['original_query']
    
    return False, None, None

def handle_follow_up_context(intent: F1QueryIntent, conversation_context: dict, user_input: str) -> F1QueryIntent:
    """Handle follow-up context inheritance"""
    is_follow_up = conversation_context['conversation_count'] > 0
    if is_follow_up and conversation_context.get('last_intent'):
        last_intent = conversation_context['last_intent']
        
        # Check if this looks like a follow-up that should inherit context
        should_inherit_context = (
            not intent.meeting and
            last_intent.meeting and
            len(user_input.split()) <= 8 and
            (
                (intent.driver1 or intent.driver2) or
                intent.team or
                any(phrase in user_input.lower() for phrase in [
                    'what about', 'how about', 'compare', 'and', 'vs', 'versus'
                ])
            )
        )
        
        if should_inherit_context:
            intent.meeting = last_intent.meeting
            intent.session_type = last_intent.session_type
            
            # Auto-detect comparison if we have multiple drivers
            if (intent.driver1 and intent.driver2) or (
                (intent.driver1 or intent.driver2) and 
                any(word in user_input.lower() for word in ['and', 'vs', 'versus', 'what about', 'how about'])
            ):
                intent.query_type = "comparison"
            elif last_intent.query_type in ["comparison", "strategy", "qualifying_results"] and intent.query_type == "general":
                intent.query_type = last_intent.query_type
    
    return intent

def interactive_f1_test():
    """Interactive testing function with conversational memory and session disambiguation"""

    print("🏎️ F1 Bot Interactive Testing Session")
    print("=" * 50)
    print("Type 'quit' to exit, 'clear' to clear memory, 'help' for examples")
    print("The bot will remember context from previous questions!")
    print("=" * 50)

    while True:
        try:
            user_input = input(f"\n[{conversation_context['conversation_count'] + 1}] You: ").strip()

            if user_input.lower() in ['quit', 'exit']:
                print("�� Thanks for testing! Goodbye!")
                break

            if user_input.lower() == 'clear':
                memory.clear()
                conversation_context.update({
                    'last_intent': None,
                    'last_question': None,
                    'conversation_count': 0,
                    'ambiguous_session_memory': None
                })
                print("🧹 Memory cleared! Starting fresh conversation.")
                continue

            if user_input.lower() == 'help':
                print_help_examples()
                continue

            is_unsupported, unsupported_response = detect_unsupported_features(user_input)
            if is_unsupported:
                print(f"\n🤖 F1 Bot: {unsupported_response}")
                continue

            # Handle clarification follow-up
            if conversation_context.get('ambiguous_session_memory'):
                success, intent, original_query = handle_clarification_response(user_input, conversation_context)
                
                if success:
                    print(f"✅ Clarified session: {intent.session_type}")
                    print("\n🤖 F1 Bot: Re-analyzing with clarified session...")
                    
                    # Get meeting key
                    meeting_key = get_meeting_key(intent.meeting) if intent.meeting else None
                    if not meeting_key:
                        print(f"❌ No meeting found for: {intent.meeting}")
                        continue

                    # Get session key with clarified session type
                    session_key = None
                    try:
                        with engine.connect() as conn:
                            session_query = f"""
                            SELECT session_key, session_name
                            FROM sessions_transformed
                            WHERE meeting_key = {meeting_key}
                            AND UPPER(session_name) = UPPER('{intent.session_type}')
                            """
                            result = conn.execute(text(session_query))
                            session_row = result.fetchone()
                            
                            if session_row:
                                session_key = session_row[0]
                            else:
                                print(f"❌ No session found for: {intent.session_type}")
                                continue
                                
                    except Exception as e:
                        print(f"❌ Error finding session: {e}")
                        continue

                    # Build SQL query with clarified intent
                    sql_query = build_f1_sql_query(intent, validator, "", original_query, clarified_session=intent.session_type)
                    
                    # Execute queries and get results
                    query_results = execute_sql_queries(sql_query)
                    data_summary = format_data_summary(query_results)
                    
                    # Generate analysis prompt with context
                    analysis_prompt = get_analysis_prompt(intent, original_query, data_summary, conversation_context, True)
                    
                    # Get response from LLM
                    messages = [
                        SystemMessage(content="You are an expert F1 analyst with conversational memory. Provide detailed, data-driven analysis. Reference previous context when relevant and maintain conversation flow."),
                        HumanMessage(content=analysis_prompt)
                    ]
                    
                    response = llm.invoke(messages)

                    # Update conversation context
                    conversation_context.update({
                        'last_intent': intent,
                        'last_question': original_query,
                        'conversation_count': conversation_context['conversation_count'] + 1
                    })
                    
                    # Print the analysis
                    print("\n📊 Analysis:")
                    print("-" * 40)
                    print(response.content)
                    print("-" * 40)

                    continue
                else:
                    print(f"❓ Clarification not recognized. Available options: {', '.join(conversation_context['ambiguous_session_memory']['options'])}")
                    conversation_context['ambiguous_session_memory'] = None

            print("\n🤖 F1 Bot: Analyzing your query...")

            # Extract intent
            intent = validator.extract_f1_entities(user_input)
            
            # Handle follow-up context inheritance
            intent = handle_follow_up_context(intent, conversation_context, user_input)
            
            # Check if this is a follow-up question
            is_follow_up = conversation_context['conversation_count'] > 0
            
            # Enhanced error handling for missing context
            if not intent.meeting:
                if is_follow_up and conversation_context.get('last_intent'):
                    print(f"🤔 I understand you're asking a follow-up question, but I need more specific information.")
                    print(f"📝 Try rephrasing with more details, such as:")
                    print(f"   • 'Could this mean Lando crashed on lap 67?'")
                    print(f"   • 'Did he have an incident in the Canadian GP?'")
                    print(f"   • 'What caused the slow lap time?'")
                    print(f"   • Or ask a new question with a specific race/driver/session")
                    continue
                else:
                    print(f"🤔 I need more specific information to help you.")
                    print(f"�� Please include:")
                    print(f"   • A specific race (e.g., 'Australian Grand Prix', 'Monaco')")
                    print(f"   • A driver name (e.g., 'Verstappen', 'Hamilton')")
                    print(f"   • A session type (e.g., 'race', 'qualifying', 'practice')")
                    print(f"   • Or try: 'help' for example queries")
                    continue

            # Get meeting key
            meeting_key = get_meeting_key(intent.meeting) if intent.meeting else None
            if not meeting_key:
                print(f"❌ I couldn't find the race '{intent.meeting}' in my database.")
                print(f"📋 Available races include:")
                available_races = validator.get_available_races()
                if available_races:
                    for race in available_races[:5]:
                        print(f"   • {race}")
                    if len(available_races) > 5:
                        print(f"   • ... and {len(available_races) - 5} more")
                else:
                    print(f"   • No races found in database")
                print(f"💡 Try using the exact race name from the list above")
                continue

            # Get session key
            session_key = None
            try:
                with engine.connect() as conn:
                    session_query = f"""
                    SELECT session_key, session_name
                    FROM sessions_transformed
                    WHERE meeting_key = {meeting_key}
                    AND UPPER(session_name) LIKE UPPER('%{intent.session_type}%')
                    """
                    result = conn.execute(text(session_query))
                    all_matches = result.fetchall()

                    if len(all_matches) > 1:
                        options = list(set([r[1] for r in all_matches]))
                        print("⚠️ Multiple session types detected:")
                        print(f"Multiple sessions match '{intent.session_type}' at {intent.meeting}: {', '.join(options)}")
                        print("Please specify which one you want:")
                        for i, option in enumerate(options, 1):
                            print(f"  {i}. {option}")
                        
                        conversation_context['ambiguous_session_memory'] = {
                            'options': options,
                            'meeting_key': meeting_key,
                            'meeting_name': intent.meeting,
                            'original_query': user_input
                        }
                        continue

                    elif len(all_matches) == 0:
                        print(f"❌ No {intent.session_type} session found for {intent.meeting}")
                        continue

                    else:
                        session_key = all_matches[0][0]
                        
            except Exception as e:
                print(f"❌ Error finding session: {e}")
                continue

            # Build SQL query
            sql_query = build_f1_sql_query(intent, validator, "", user_input)
            
            # Check if the SQL query returned a structured clarification response
            if isinstance(sql_query, dict) and sql_query.get("type") == "clarification_needed":
                print("⚠️ Multiple session types detected:")
                print(sql_query["message"])
                print("Please specify which one you want:")
                for i, option in enumerate(sql_query["options"], 1):
                    print(f"  {i}. {option}")
                
                conversation_context['ambiguous_session_memory'] = {
                    'options': sql_query['options'],
                    'meeting_key': sql_query['meeting_key'],
                    'meeting_name': sql_query['meeting_name'],
                    'original_query': user_input
                }
                continue
            
            # Execute queries and get results
            query_results = execute_sql_queries(sql_query)
            data_summary = format_data_summary(query_results)
            
            # Generate analysis prompt with context
            analysis_prompt = get_analysis_prompt(intent, user_input, data_summary, conversation_context, is_follow_up)
            
            # Add context awareness for follow-up questions
            if is_follow_up and conversation_context.get('last_question'):
                analysis_prompt += f"""
                
                CONVERSATION CONTEXT:
                - Previous question: "{conversation_context['last_question']}"
                - This is question #{conversation_context['conversation_count'] + 1} in our conversation
                - The user may be asking for similar analysis but for different races/drivers
                - Provide consistent analysis style but for the new query
                """
            
            # Get response from LLM
            messages = [
                SystemMessage(content="You are an expert F1 analyst with conversational memory. Provide detailed, data-driven analysis. Reference previous context when relevant and maintain conversation flow."),
                HumanMessage(content=analysis_prompt)
            ]
            
            response = llm.invoke(messages)
            
            # Update conversation context
            conversation_context.update({
                'last_intent': intent,
                'last_question': user_input,
                'conversation_count': conversation_context['conversation_count'] + 1
            })
            
            # Print the analysis
            print("\n📊 Analysis:")
            print("-" * 40)
            print(response.content)
            print("-" * 40)

        except Exception as e:
            print(f"❌ Error: {e}")
            continue

def print_help_examples():
    """Print example queries for testing"""
    print("\n🏎️ Example Queries to Test:")
    print("=" * 40)
    
    examples = [
        "🏁 Race Results:",
        "  • Who won the Australian Grand Prix?",
        "  • What were the race results in Monaco?",
        "  • Who finished on the podium at the British Grand Prix?",
        
        "\n🏎️ Driver Comparisons:",
        "  • Compare Verstappen and Hamilton in the Miami race",
        "  • How did Norris and Piastri perform in qualifying?",
        "  • Compare the lap times between Leclerc and Sainz",
        
        "\n��️ Strategy Analysis:",
        "  • What strategy did Red Bull use in the race?",
        "  • How many pit stops did Mercedes make?",
        "  • What tire compounds did Ferrari use?",
        
        "\n⚠️ Incident Analysis:",
        "  • What happened to Norris on lap 67 of the canadian GP?",
        "  • Why was Hamilton slow on lap 23?",
        "  • What caused the incident on lap 8?",
        
        "\n🏆 Qualifying:",
        "  • Who got pole position in qualifying?",
        "  • Show me the qualifying results for the Australian GP",
        "  • What were the fastest qualifying times?",
        
        "\n🔄 Follow-up Questions (test context):",
        "  • How about the British Grand Prix?",
        "  • Compare them in the race instead",
        "  • What about their tire strategy?",
        "  • Show me the same analysis for Monaco"
    ]
    
    for example in examples:
        print(example)
    
    print("\n💡 Tips:")
    print("  • The bot remembers previous questions")
    print("  • Use 'clear' to reset conversation memory")
    print("  • Try follow-up questions to test context awareness")

# Start the interactive testing
if __name__ == "__main__":
    interactive_f1_test()

🏎️ F1 Bot Interactive Testing Session
Type 'quit' to exit, 'clear' to clear memory, 'help' for examples
The bot will remember context from previous questions!

🤖 F1 Bot: Analyzing your query...
�� DEBUG: Final query_type: comparison

📊 Analysis:
----------------------------------------
Based on the data provided from the Miami Grand Prix race between Max Verstappen and Lewis Hamilton, let's delve into a detailed comparison of their performance:

1. **Average Lap Time**:
   - Max Verstappen had an average lap time of 01:33.614, while Lewis Hamilton's average lap time was slightly slower at 01:33.912. Verstappen was quicker on average by around 0.3 seconds per lap.

2. **Best and Worst Laps**:
   - Both drivers set their best lap at 01:30.00, showcasing their speed potential. However, Hamilton's worst lap was 135.0 seconds compared to Verstappen's 125.0 seconds, indicating Verstappen had better consistency in lap times.

3. **Consistency**:
   - Verstappen demonstrated better consistency